In [1]:
import folium
import gradio as gr

def generate_map(lat, lon):
    print(f"lat: {lat}, lon: {lon}")
    
    # 创建地图，中心点为输入的坐标
    m = folium.Map(
        location=[lat, lon],
        zoom_start=15,  # 设置初始缩放级别
        tiles='OpenStreetMap'  # 使用 OpenStreetMap 底图
    )
    
    # 添加标记
    folium.Marker(
        location=[lat, lon],
        popup=f"位置: {lat}, {lon}",  # 点击标记时显示的信息
        icon=folium.Icon(color='red')  # 设置标记样式
    ).add_to(m)
    
    # 保存到临时HTML文件
    map_html = m._repr_html_()
    return map_html

# 创建 Gradio 接口
interface = gr.Interface(
    fn=generate_map,
    inputs=[
        gr.Number(label="Latitude", value=31.3385, precision=8),
        gr.Number(label="Longitude", value=121.5020, precision=8)
    ],
    outputs=gr.HTML(),  # 使用 HTML 输出
    cache_examples=False
)

interface.launch()

e:\zfy\anaconda3\envs\swarm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


lat: 31.3385, lon: 121.502


In [1]:
import folium
import gradio as gr
from itertools import permutations

import math

def calculate_distance(point1, point2):
    """计算两个点之间的距离，以米为单位"""
    # 将纬度和经度从度转换为弧度
    lat1, lon1 = map(math.radians, point1)
    lat2, lon2 = map(math.radians, point2)
    
    # Haversine公式
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.asin(math.sqrt(a))
    
    # 地球的平均半径，单位为米
    r = 6371000
    return c * r

def find_shortest_path(start, end, waypoints):
    """找到从起点到终点经过所有途经点的最短路径"""
    all_points = [start] + waypoints + [end]
    waypoints_permutations = permutations(waypoints)
    shortest_path = None
    shortest_distance = float('inf')
    
    for perm in waypoints_permutations:
        current_path = [start] + list(perm) + [end]
        current_distance = sum(calculate_distance(current_path[i], current_path[i+1]) for i in range(len(current_path) - 1))
        
        if current_distance < shortest_distance:
            shortest_distance = current_distance
            shortest_path = current_path
    
    return shortest_path, shortest_distance

def generate_map_with_path(start, end, waypoints):
    # 创建地图，中心点为起始点
    m = folium.Map(
        location=start,
        zoom_start=18,
        tiles='OpenStreetMap'
    )
    
    # 计算最短路径
    shortest_path, _ = find_shortest_path(start, end, waypoints)
    print(_)
    
    # 在地图上绘制路径
    folium.PolyLine(locations=shortest_path, color='blue').add_to(m)
    
    # 添加起始点和终止点标记
    folium.Marker(location=start, popup="起始点", icon=folium.Icon(color='green')).add_to(m)
    folium.Marker(location=end, popup="终止点", icon=folium.Icon(color='red')).add_to(m)
    
    # 添加途经点标记
    for waypoint in waypoints:
        folium.Marker(location=waypoint, popup="途经点", icon=folium.Icon(color='orange')).add_to(m)
    
    map_html = m._repr_html_()
    return map_html

def parse_points(points_str):
    """解析输入的坐标字符串为列表"""
    points = []
    for point_str in points_str.split(';'):
        lat, lon = map(float, point_str.split(','))
        points.append((lat, lon))
    return points

# 创建 Gradio 接口
interface = gr.Interface(
    fn=lambda start, end, waypoints: generate_map_with_path(
        parse_points(start)[0],
        parse_points(end)[0],
        parse_points(waypoints)
    ),
    inputs=[
        gr.Textbox(label="起始点 (格式: lat,lon)",value="31.3385, 121.5020"),
        gr.Textbox(label="终止点 (格式: lat,lon)",value="31.3389, 121.5025"),
        gr.Textbox(label="途经点 (格式: lat,lon;lat,lon;...)", value="31.3384, 121.5019; 31.3387, 121.5018; 31.3384, 121.5023")
    ],
    outputs=gr.HTML(),
    cache_examples=False
)

interface.launch()

e:\zfy\anaconda3\envs\swarm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


160.67240031784797
